In [278]:
###### Importando Bibliotecas:

import sys 
import os 
sys.path.append('../../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

from sklearn.cluster import KMeans
import warnings
import os
warnings.filterwarnings("ignore")

from src.utils import utilidades as myutils


parameters = ['coliformes', 'od', 'turbidez', 'fosforo', 'solido', 'dbo', 'temperatura', 'ph']

# for parameter in parameters:
#     # Carregando a base de dados:
#     df = pd.read_csv('../../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
#     # Número de linhas e colunas:
#     df.shape
#     # Estatística Descritiva:
#     df.describe()

#     # Tipos de Dados:
#     df.dtypes

#     # Definindo um estilo para os gráficos:
#     plt.style.use('fivethirtyeight')

#     # Verificando as distribuição dos dados:
#     plt.figure(4 , figsize = (10 , 6))
#     n = 0 
#     for x in ['valor' , 'Altitude']:
        
#         sns.distplot(df[x] , bins = 25)
#         plt.title(x +' ' + parameter)
    
    
    
#         # Contagem de Amostras por UGRHI:

#     sns.countplot(y = 'UGRHI' , data = df)
    
    
#         # Renda Anual vs Pontuação de Gastos:    
#     for ugrhi in ['10 - SOROCABA/MEDIO TIETE' , '06 - ALTO TIÊTE','05 - PIRACICABA/CAPIVARI/JUNDIAI','07 - BAIXADA SANTISTA','02 - PARAIBA DO SUL']:
#         plt.scatter(x = 'Altitude',y = 'valor' , data = df[df['UGRHI'] == ugrhi] ,s = 50 , alpha = 0.5 , label = ugrhi)
#     plt.xlabel('Altitude'), plt.ylabel('Valor pH') 
#     #plt.title('Altitude vs Valor pH')
#     plt.legend()
#     plt.show()

In [311]:

def clean_group_data(dataset, parameter, range_upper = 1,range_lower = 1):
    
    q1, q3 = np.percentile(dataset[parameter],[25,75])
    iqr = q3 - q1
    lower_bound = q3 - range_upper*(1.5 * iqr)  
    upper_bound = q3 + range_upper*(1.5 * iqr)  

    # Deleting lower bound and upper bound from the dataset LinkTT2
    dataset = DFmerged.loc[(dataset[parameter] >= lower_bound) & 
                                              (dataset[parameter] <= upper_bound)]
    
    return dataset

In [325]:
from sklearn.preprocessing import MinMaxScaler

DFmerge = pd.DataFrame()
for parameter in parameters:
    # Concatenating all the parameter files
    df = pd.read_csv('../../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
    DFmerge = pd.concat([DFmerge, df])


for parameter in DFmerge['parametro'].unique():    
    DFmerged = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02050') &
                      (DFmerge['parametro'] == parameter)]  

    

    # DFmerge.groupby([pd.Grouper(freq='1M'), 'codigo_ponto']).mean().unstack()
    # DFmerge.groupby(['codigo_ponto', 'UGRHI'])['valor'].count().unstack()

    DFmerged = DFmerged[DFmerged['codigo_ponto'] == 'TIET02050']    
    DFmerged = clean_group_data(DFmerged, 'valor')
    
    DFmerged['data_coleta'] = pd.to_datetime(DFmerged['data_coleta'])
    DFmerged.index = DFmerged['data_coleta']    
    DFmerged = DFmerged.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    
    scaler = MinMaxScaler()
#     DFmerged = scaler.fit_transform()

    # #Mostra a quantidade de NaN no dataframe
    # for field in DFmerge.columns:
    #     print(field, 'NaN:', DFmerge[field].isnull().sum())

    X  = DFmerged[:-1]
    y  = DFmerged[1:]


    dataset = pd.DataFrame(np.concatenate([X,y], axis=1))
    dataset.columns = ['X','y']
    dataset.dropna(inplace=True)

    train_X = np.array(dataset['X']).reshape(len(dataset), 1)
    train_y = np.log(np.array(dataset['y']).reshape(len(dataset), 1) + 0.1)

    model = LinearRegression(normalize=False)


    model.fit(train_X[:-40],train_y[:-40])
    score = model.score(train_X,train_y)
#     print(score)

    #Dados de teste
    previsoes = model.predict(train_X[-40:])



    print(parameter, len(DFmerged),'MAPE',
          str(myutils.mean_absolute_percentage_error(train_y[-40:], previsoes)))



Coliformes Termotolerantes 152 MAPE 35.1738819530236
Oxigênio Dissolvido 205 MAPE 24.609556044046172
Turbidez 190 MAPE 47.102965888212296
Fósforo Total 193 MAPE 7.496568255115281
Sólido Total 206 MAPE 6.93318136986519
DBO (5, 20) 162 MAPE 7.112216551706887
Temperatura da Água 196 MAPE 2.563773292097012
pH 180 MAPE 1.5691035852509405
